<a href="https://colab.research.google.com/github/aditya31tiwari/FAKE_NEWS_DETECTION/blob/main/fnd_multi_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing the libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import joblib

In [ ]:
#pre-processing the FakeNewsNet dataset
#(4csv files, BuzzFeed_fake&true PoliticalFact_fake&true)
from google.colab import files
print("Upload the CSV files first")
uploaded = files.upload()


Upload the CSV files first


Saving BuzzFeed_fake_news_content.csv to BuzzFeed_fake_news_content.csv
Saving BuzzFeed_real_news_content.csv to BuzzFeed_real_news_content.csv
Saving PolitiFact_fake_news_content.csv to PolitiFact_fake_news_content.csv
Saving PolitiFact_real_news_content.csv to PolitiFact_real_news_content.csv


In [ ]:
#preprocessing isot
print("Upload ISOT dataset:")
uploaded = files.upload()


Upload ISOT dataset:


Saving Fake.csv to Fake (1).csv
Saving True.csv to True (1).csv


In [ ]:
bf_fake = pd.read_csv("BuzzFeed_fake_news_content.csv")
bf_real = pd.read_csv("BuzzFeed_real_news_content.csv")
pf_fake = pd.read_csv("PolitiFact_fake_news_content.csv")
pf_real = pd.read_csv("PolitiFact_real_news_content.csv")

true = pd.read_csv("True.csv")
fake = pd.read_csv("Fake.csv")

bf_fake["label"] = 0
bf_real["label"] = 1
pf_fake["label"] = 0
pf_real["label"] = 1

true["label"] = 1
fake["label"] = 0



In [ ]:
from google.colab import files
print("Upload the Misinformation & Fake News Dataset")
uploaded = files.upload()

Upload the Misinformation & Fake News Dataset


Saving DataSet_Misinfo_TRUE.csv to DataSet_Misinfo_TRUE (1).csv
Saving DataSet_Misinfo_FAKE.csv to DataSet_Misinfo_FAKE (1).csv


In [ ]:
#Misinformation Dataset cleaning
misinfo_fake = pd.read_csv("DataSet_Misinfo_FAKE.csv")
misinfo_true = pd.read_csv("DataSet_Misinfo_TRUE.csv")

# Add Labels
misinfo_fake["label"] = 0
misinfo_true["label"] = 1

#Merge into one DataFrame
misinfo = pd.concat([misinfo_fake, misinfo_true], axis=0)

#keep only the text + label columns
if "text" in misinfo.columns:
  misinfo = misinfo[["text","label"]]
elif "content" in misinfo.columns:
  misinfo = misinfo.rename(columns={"content": "text"})[["text","label"]]

#Drop empty/duplicates
misinfo = misinfo.dropna().drop_duplicates()

print("Misinfo Dataset Shape:", misinfo.shape)
print(misinfo['label'].value_counts())
print(misinfo.head())


Misinfo Dataset Shape: (68604, 2)
label
1    34526
0    34078
Name: count, dtype: int64
                                                text  label
0  Donald Trump just couldn t wish all Americans ...      0
1  House Intelligence Committee Chairman Devin Nu...      0
2  On Friday, it was revealed that former Milwauk...      0
3  On Christmas day, Donald Trump announced that ...      0
4  Pope Francis used his annual Christmas Day mes...      0


In [ ]:
#ISOT Dataset Cleaning
true["label"] = 1
fake["label"] = 0

# Combine title + text for richer content
true["text"] = true["title"].fillna("") + " " + true["text"].fillna("")
fake["text"] = fake["title"].fillna("") + " " + fake["text"].fillna("")

# Keep only text + label
isot = pd.concat([true[["text","label"]], fake[["text","label"]]], axis=0).dropna().drop_duplicates()

print("ISOT dataset shape:", isot.shape)
print(isot['label'].value_counts())
print(isot.head())

ISOT dataset shape: (39105, 2)
label
1    21197
0    17908
Name: count, dtype: int64
                                                text  label
0  As U.S. budget fight looms, Republicans flip t...      1
1  U.S. military to accept transgender recruits o...      1
2  Senior U.S. Republican senator: 'Let Mr. Muell...      1
3  FBI Russia probe helped by Australian diplomat...      1
4  Trump wants Postal Service to charge 'much mor...      1


In [ ]:
# FakeNewsNet: BuzzFeed + PolitiFact (already loaded earlier as bf_fake, bf_real, pf_fake, pf_real)

# Function to normalize FakeNewsNet datasets
def prepare_fnn(df):
    if "title" in df.columns and "text" in df.columns:
        df["text"] = df["title"].fillna("") + " " + df["text"].fillna("")
    elif "content" in df.columns:
        df = df.rename(columns={"content": "text"})
    return df[["text","label"]].dropna()

bf_fake = prepare_fnn(bf_fake)
bf_real = prepare_fnn(bf_real)
pf_fake = prepare_fnn(pf_fake)
pf_real = prepare_fnn(pf_real)

fakenewsnet = pd.concat([bf_fake, bf_real, pf_fake, pf_real], axis=0).drop_duplicates()

print("FakeNewsNet shape:", fakenewsnet.shape)
print(fakenewsnet['label'].value_counts())
print(fakenewsnet.head())


FakeNewsNet shape: (403, 2)
label
1    203
0    200
Name: count, dtype: int64
                                                text  label
0  Proof The Mainstream Media Is Manipulating The...      0
1  Charity: Clinton Foundation Distributed “Water...      0
2  A Hillary Clinton Administration May be Entire...      0
3  Trump’s Latest Campaign Promise May Be His Mos...      0
4  Website is Down For Maintenance Website is Dow...      0


In [ ]:
# Merge everything together
data = pd.concat([misinfo, isot, fakenewsnet], axis=0)

# Drop NAs and duplicates, shuffle
data = data.dropna().drop_duplicates().sample(frac=1).reset_index(drop=True)

print("Final merged dataset shape:", data.shape)
print(data['label'].value_counts())
print(data.head())


Final merged dataset shape: (108112, 2)
label
1    55926
0    52186
Name: count, dtype: int64
                                                text  label
0  CHARLIE DANIELS WARNS LIBERALS: Knock It Off O...      0
1  EPA actions in Flint water crisis to be examin...      1
2  GRIFTER HILLARY CLINTON WAS PAID For Speeches ...      0
3  AURORA, Colo.  —   It took 11 days of calling ...      1
4  The largest and busiest airport in Arkansas wo...      1


In [ ]:
#Saving the Cleaned and Final Dataset
data.to_csv("final_merged_dataset.csv", index=False)

from google.colab import files
files.download("final_merged_dataset.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>